In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import hashlib
%matplotlib inline

In [11]:
date = pd.read_hdf("../hdf/train_date_L1_normalized.hdf")
c = [x for x in date.columns if "S24" in x]
date = date.loc[:, c]

In [12]:
categorical = pd.read_hdf("../hdf/train_categorical.hdf")
c = [x for x in categorical.columns if "S24" in x]
categorical = categorical.loc[:, c]

In [13]:
numeric = pd.read_hdf("../hdf/train_numeric.hdf")
c = [x for x in numeric.columns if "S24" in x]
numeric = numeric.loc[:, c]

In [26]:
response = pd.read_hdf("../hdf/train_response.hdf")

In [47]:
hash_df = pd.DataFrame(index=response.index, columns=[])

In [50]:
binary_date = date.fillna(-1.0)
binary_date.where(binary_date == -1.0, 1.0, inplace=True)
binary_date.where(binary_date == 1.0, np.nan, inplace=True)

In [56]:
hash_df["S24_date_md5"] = binary_date.apply(
    lambda x: int(hashlib.md5(x.values).hexdigest(), 16), axis=1)

In [63]:
hash_df = hash_df.astype(np.float32)
hash_df = pd.concat([hash_df, response], axis=1)

In [67]:
df = pd.DataFrame(
    index=hash_df[hash_df["Response"] ==
                  0.0]["S24_date_md5"].value_counts().index,
    columns=[])
df["count0"] = hash_df[hash_df["Response"] ==
                       0.0]["S24_date_md5"].value_counts()
df["count1"] = hash_df[hash_df["Response"] ==
                       1.0]["S24_date_md5"].value_counts()
df["count1"] / df.T.sum()

In [93]:
categorical = pd.read_hdf("../hdf/train_categorical.hdf")
c = [x for x in categorical.columns if "S24" in x]
categorical = categorical.loc[:, c]

In [107]:
categorical_hash = pd.DataFrame(index=categorical.index, columns=[])
categorical_hash["S24_C_md5"] = categorical.apply(
    lambda x: int(hashlib.md5(x.values).hexdigest(), 16), axis=1)
categorical_hash = pd.concat([categorical_hash, response], axis=1)
categorical_hash = categorical_hash.astype(np.float32)

In [112]:
df = pd.DataFrame(columns=[])
df["count0"] = categorical_hash[categorical_hash["Response"] ==
                                0.0]["S24_C_md5"].value_counts()
df["count1"] = categorical_hash[categorical_hash["Response"] ==
                                1.0]["S24_C_md5"].value_counts()
df["ratio"] = df["count1"] / df.T.sum()
df.sort_values(["count0"], ascending=[0])
i1 = df.index[1]
i2 = df.index[2]

In [88]:
categorical_hash[["S24_C_md5"]].to_hdf(
    "../hdf/train_S24_C_md5.hdf", "df", mode="w")